In [ ]:
#Check GPU availability

import torch

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, using CPU")

GPU is available


TimesFM from Google

*   https://github.com/google-research/timesfm
*   https://huggingface.co/google/timesfm-1.0-200m



In [ ]:
!git clone https://github.com/google-research/timesfm

fatal: destination path 'timesfm' already exists and is not an empty directory.


In [ ]:
!pip install einshape praxis paxml utilsforecast


In src/timesfm.py
https://github.com/google-research/timesfm/issues/13

from praxis.layers import transformers

import sys

sys.path.append(path.dirname(path.abspath(__file__))) #__file__ atentie: underscore underscore file underscore underscore

import patched_decoder

In [ ]:
import timesfm

tfm = timesfm.TimesFm(
    context_len=96, #<context>,
    horizon_len=96,  #<horizon>,
    input_patch_len=32,
    output_patch_len=128,
    num_layers=20,
    model_dims=1280,
    #backend=<backend>, #cpu gpu
    backend="gpu"
)
tfm.load_from_checkpoint(repo_id="google/timesfm-1.0-200m")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

(…)oint_1100000/descriptor/descriptor.pbtxt:   0%|          | 0.00/499 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.73k [00:00<?, ?B/s]

checkpoint:   0%|          | 0.00/814M [00:00<?, ?B/s]

...

.gitattributes:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

Constructing model weights.


Constructed model weights in 3.45 seconds.
Restoring checkpoint from /root/.cache/huggingface/hub/models--google--timesfm-1.0-200m/snapshots/8775f7531211ac864b739fe776b0b255c277e2be/checkpoints.


ERROR:absl:For checkpoint version > 1.0, we require users to provide
          `train_state_unpadded_shape_dtype_struct` during checkpoint
          saving/restoring, to avoid potential silent bugs when loading
          checkpoints to incompatible unpadded shapes of TrainState.


Restored checkpoint in 1.43 seconds.
Jitting decoding.
Jitted decoding in 19.53 seconds.


## Rolling window evaluation metrics ##

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Greece.csv to Greece.csv


In [ ]:
import pandas as pd

# Assuming 'yourfile.csv' is the name of your uploaded file
df = pd.read_csv('Greece.csv')

# Display the DataFrame
print(df.tail())

      Country ISO3 Code       Datetime (UTC)     Datetime (Local)  \
81067  Greece       GRC  2024-03-31 19:00:00  2024-03-31 22:00:00   
81068  Greece       GRC  2024-03-31 20:00:00  2024-03-31 23:00:00   
81069  Greece       GRC  2024-03-31 21:00:00  2024-04-01 00:00:00   
81070  Greece       GRC  2024-03-31 22:00:00  2024-04-01 01:00:00   
81071  Greece       GRC  2024-03-31 23:00:00  2024-04-01 02:00:00   

       Price (EUR/MWhe)  
81067             65.81  
81068             60.90  
81069             48.07  
81070             43.45  
81071             37.15  


In [ ]:
df = df[['Datetime (UTC)', 'Country', 'Price (EUR/MWhe)']] #trebuie sa redenumim coloanele ca sa mearga ; in plus fiecare serie trebueie sa
# aiba un unique id (aici doar 0 pt. ca e numai o singura serie). Daca folosim all_countries.csv, atuncy unique_id poate fy country
df.columns = ['ds', 'unique_id', 'y']
input_df=df
# Convert the 'time' column to datetime
input_df['ds'] = pd.to_datetime(input_df['ds'])

<ipython-input-7-17fb1fda0b2d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df['ds'] = pd.to_datetime(input_df['ds'])


In [ ]:
print(input_df.tail())

                       ds unique_id      y
81067 2024-03-31 19:00:00    Greece  65.81
81068 2024-03-31 20:00:00    Greece  60.90
81069 2024-03-31 21:00:00    Greece  48.07
81070 2024-03-31 22:00:00    Greece  43.45
81071 2024-03-31 23:00:00    Greece  37.15


## With for (rolling window) ##

In [ ]:
import numpy as np
import torch
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pickle

# Assuming `pipeline` is your Chronos pipeline and `df` is your DataFrame

# Function to calculate RMSE
def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Initialize lists to store predictions, actual values, and window metrics
predictions = []
actuals = []
window_mae = []
window_rmse = []
window_mse = []
window_predictions = []  # To store the median predictions for each window
window_actuals = []      # To store the actual values for each window

# Rolling window parameters
input_length = 96
prediction_length = 96
# num_samples = 1 #32

# Get the last 3000 rows of the DataFrame
df_last_3000 = input_df.iloc[-3000:]

# Initialize the iteration counter
nb_iter = 1

# Iterate over the last 3000 rows using a rolling window
for start_idx in range(len(df_last_3000) - input_length - prediction_length + 1):
    # Define the end index for the input and prediction window
    end_idx = start_idx + input_length
    pred_end_idx = end_idx + prediction_length

    # Extract the input sequence
    # input_sequence = df_last_3000["y"].to_numpy()[start_idx:end_idx]
    # rolling_df = input_df.iloc[i:i + prediction_length]
    input_sequence = df_last_3000.iloc[start_idx:end_idx]
    # Extract the actual prediction sequence
    actual_sequence = df_last_3000["y"].to_numpy()[end_idx:pred_end_idx] #ok as of now

    # Convert input sequence to tensor
    # input_tensor = torch.tensor(input_sequence, dtype=torch.float32)

     # Forecast on the rolling_df
    forecast_df = tfm.forecast_on_df(
            inputs=input_sequence,
            freq="H",  # Assuming hourly frequency
            value_name="y",
            num_jobs=-1,
    )

    # Store the median predictions (each sample over samples)
    predictionsTFM=forecast_df['timesfm'].values
    predictions.append(predictionsTFM)
    actuals.append(actual_sequence)

    # Store window predictions and actuals
    window_predictions.append(predictionsTFM)
    window_actuals.append(actual_sequence)

    # Calculate metrics using median predictions for the current window
    window_mae.append(mean_absolute_error(actual_sequence, predictionsTFM))
    window_rmse.append(root_mean_squared_error(actual_sequence, predictionsTFM))
    window_mse.append(mean_squared_error(actual_sequence, predictionsTFM))

    # Print the current iteration number with prefix 'nb'
    print(f"nb{nb_iter}")
    nb_iter=nb_iter+1

# Convert lists to numpy arrays for easier metric calculation
predictions = np.array(predictions)
actuals = np.array(actuals)

# Calculate global performance metrics as the mean of window metrics
global_mae = np.mean(window_mae)
global_rmse = np.mean(window_rmse)
global_mse = np.mean(window_mse)

# Store predictions and metrics in a dictionary
results = {
    "predictions": predictions,
    "actuals": actuals,
    "window_predictions": window_predictions,
    "window_actuals": window_actuals,
    "metrics": {
        "Global MAE": global_mae,
        "Global RMSE": global_rmse,
        "Global MSE": global_mse
    }
}

# Print the global metrics
print("Global MAE:", global_mae)
print("Global RMSE:", global_rmse)
print("Global MSE:", global_mse)

#To save: results

# Save dictionary as a pickle file
with open('results_TimesFM_GPU_Greece.pkl', 'wb') as pickle_file:
    pickle.dump(results, pickle_file)

# Download the file to your local machine
files.download(f'results_TimesFM_GPU_Greece.pkl')

Streaming output truncated to the last 5000 lines.
nb1560
Processing dataframe with multiple processes.
Finished preprocessing dataframe.
Finished forecasting.
nb1561
Processing dataframe with multiple processes.
Finished preprocessing dataframe.
Finished forecasting.
nb1562
Processing dataframe with multiple processes.
Finished preprocessing dataframe.
Finished forecasting.
nb1563
Processing dataframe with multiple processes.
Finished preprocessing dataframe.
Finished forecasting.
nb1564
Processing dataframe with multiple processes.
Finished preprocessing dataframe.
Finished forecasting.
nb1565
Processing dataframe with multiple processes.
Finished preprocessing dataframe.
Finished forecasting.
nb1566
Processing dataframe with multiple processes.
Finished preprocessing dataframe.
Finished forecasting.
nb1567
Processing dataframe with multiple processes.
Finished preprocessing dataframe.
Finished forecasting.
nb1568
Processing dataframe with multiple processes.
Finished preprocessing d

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>